In [1]:
import os
import pandas as pd
import numpy as np
import joblib
from rdkit import Chem
from rdkit.Chem import MACCSkeys, AllChem, rdMolDescriptors, Descriptors
from sklearn.preprocessing import QuantileTransformer
from sklearn.impute import SimpleImputer
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.preprocessing import QuantileTransformer
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

In [2]:
# ----- 1. import drug data 
drug_info = pd.read_csv('drug_info_update.tsv', sep='|', header=None)
drug_info.columns = ['Drugbank ID','PubChem CID','Canonical SMILES','PubchemFP','ECFP']
drug_info.info() # 1156
drug_info.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1156 entries, 0 to 1155
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Drugbank ID       1156 non-null   object 
 1   PubChem CID       1156 non-null   float64
 2   Canonical SMILES  1156 non-null   object 
 3   PubchemFP         1156 non-null   object 
 4   ECFP              1156 non-null   object 
dtypes: float64(1), object(4)
memory usage: 45.3+ KB


,Drugbank ID,PubChem CID,Canonical SMILES,PubchemFP,ECFP
0,DB00014,5311128.0,CC(C)CC(C(=O)NC(CCCN=C(N)N)C(=O)N1CCCC1C(=O)NN...,0000000000000000000000110111000111110000011111...,0100000000000000000010000000000000000000000000...
1,DB00035,5311065.0,C1CC(N(C1)C(=O)C2CSSCCC(=O)NC(C(=O)NC(C(=O)NC(...,0000000000000000000000110111000111110000011111...,0100000010000000000000000000000010000000010000...
2,DB00050,25074887.0,CC(C)CC(C(=O)NC(CCCN=C(N)N)C(=O)N1CCCC1C(=O)NC...,0000000000000000000000110111000111110000011111...,0100000000000000000001000000000010000000000000...
3,DB00091,5284373.0,CCC1C(=O)N(CC(=O)N(C(C(=O)NC(C(=O)N(C(C(=O)NC(...,0000000000000000000000110111000111110000011111...,0100010000000000000100000000000000000010000000...
4,DB00104,448601.0,CC(C1C(=O)NC(CSSCC(C(=O)NC(C(=O)NC(C(=O)NC(C(=...,0000000000000000000000110111000111110000011111...,0100000000000000000000000000000010000000000000...


In [3]:
# ----- 2.import feature data
parent_path = os.path.dirname(os.getcwd())
feature_data = pd.read_csv(os.path.join(parent_path,'feature_data.tsv'), sep='|')
feature_data = feature_data.drop_duplicates(subset="PubChem CID", keep="first")
feature_data = feature_data.dropna(subset=['PubChem CID'])
feature_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 109817 entries, 0 to 109816
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   TAID              109817 non-null  object 
 1   Name              14648 non-null   object 
 2   IUPAC Name        104293 non-null  object 
 3   PubChem CID       109817 non-null  float64
 4   Canonical SMILES  109817 non-null  object 
 5   InChIKey          109736 non-null  object 
 6   ECFP2             109817 non-null  object 
 7   ECFP4             109817 non-null  object 
 8   ECFP6             109817 non-null  object 
 9   MACCS             109817 non-null  object 
 10  Morgan            109817 non-null  object 
 11  Rdkit2D           109817 non-null  object 
dtypes: float64(1), object(11)
memory usage: 10.9+ MB


In [4]:
# ----- 3.merge data
drug_info['PubChem CID'] = drug_info['PubChem CID'].astype(float)
drug_info_updated = drug_info.iloc[:,0:2].merge(feature_data.iloc[:,3:], how="left", on="PubChem CID")
drug_info_updated.info()
"""
    note:
        1.SMILES/ECFP/MACCS/Morgan has 238 missing valuse
        2. RDkit2D is a 200-length vector rather than 208-lenth?
"""

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1156 entries, 0 to 1155
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Drugbank ID       1156 non-null   object 
 1   PubChem CID       1156 non-null   float64
 2   Canonical SMILES  918 non-null    object 
 3   InChIKey          917 non-null    object 
 4   ECFP2             918 non-null    object 
 5   ECFP4             918 non-null    object 
 6   ECFP6             918 non-null    object 
 7   MACCS             918 non-null    object 
 8   Morgan            918 non-null    object 
 9   Rdkit2D           918 non-null    object 
dtypes: float64(1), object(9)
memory usage: 99.3+ KB


'\n    note:\n        1.SMILES/ECFP/MACCS/Morgan has 238 missing valuse\n        2. RDkit2D is a 200-length vector rather than 208-lenth?\n'

In [5]:
drug_info_updated.head()

,Drugbank ID,PubChem CID,Canonical SMILES,InChIKey,ECFP2,ECFP4,ECFP6,MACCS,Morgan,Rdkit2D
0,DB00014,5311128.0,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,BLCLNMBMMGCOAS-URPVMXJPSA-N,0100000000000000000000000000000000000000000000...,0100001100000000000010000000000000000000000000...,0100001100000000000010000000000000000000000000...,0000000000000000000000000100000000000100000100...,0100100000000000000000000000000001000000000000...,0.048262902，0.999993637，0.999290769，0.99891407...
1,DB00035,5311065.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DB00050,25074887.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DB00091,5284373.0,C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC...,PMATZTZNYRCHOR-CGLBZJNRSA-N,0100010000000000000100000000000000000010000000...,0100010000000000000100000000000000000010000000...,0100010000000000000100000000000000000011000000...,0000000000000000000000000000000000000000000000...,0100010000100000000100000000000001000010000000...,0.996694664，0.998144505，0.999254951，0.99933621...
4,DB00104,448601.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
# ----- 4. check missing value
def simple_fingerprint_imputation(df, smiles_col='Canonical SMILES'):
    """calculate missing fingerprints"""
    df = df.copy()
    
    # 1. calculate ECFP
    if 'ECFP2' in df.columns:
        missing_mask = df['ECFP2'].isna() | (df['ECFP2'] == '')
        print(f"fill {missing_mask.sum()} missing ECFP2")
        
        for idx in df[missing_mask].index:
            smiles = str(df.at[idx, smiles_col])
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                fp2 = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
                df.at[idx, 'ECFP2'] = ''.join(['1' if fp2.GetBit(i) else '0' for i in range(2048)])
                fp4 = AllChem.GetMorganFingerprintAsBitVect(mol, radius=4, nBits=2048)
                df.at[idx, 'ECFP4'] = ''.join(['1' if fp4.GetBit(i) else '0' for i in range(2048)])
                fp6 = AllChem.GetMorganFingerprintAsBitVect(mol, radius=6, nBits=2048)
                df.at[idx, 'ECFP6'] = ''.join(['1' if fp6.GetBit(i) else '0' for i in range(2048)])
   

    # 2. calculate MACCS (168)
    if 'MACCS' in df.columns:
        missing_mask = df['MACCS'].isna() | (df['MACCS'] == '')
        print(f"fill {missing_mask.sum()} missing MACCS")
        
        for idx in df[missing_mask].index:
            smiles = str(df.at[idx, smiles_col])
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                maccs_fp = MACCSkeys.GenMACCSKeys(mol)
                df.at[idx, 'MACCS'] = ''.join(['1' if maccs_fp.GetBit(i) else '0' for i in range(167)])
                df.at[idx, 'MACCS'] = '0' + df.at[idx, 'MACCS']
                
    # 3. calculate Morgan (1024)
    if 'Morgan' in df.columns:
        missing_mask = df['Morgan'].isna() | (df['Morgan'] == '')
        print(f"fill {missing_mask.sum()} missing Morgan")
        
        for idx in df[missing_mask].index:
            smiles = str(df.at[idx, smiles_col])
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)
                df.at[idx, 'Morgan'] = ''.join(['1' if fp.GetBit(i) else '0' for i in range(1024)])
    
    print("Finished!")
    return df

# drug_info_updated = simple_fingerprint_imputation(drug_info_updated)
# drug_info_updated.info()

print("\n指纹长度验证:")
print(f"ECFP2: {len(drug_info_updated['ECFP2'].iloc[0])} (应为2048)")
print(f"ECFP4: {len(drug_info_updated['ECFP4'].iloc[0])} (应为2048)")
print(f"ECFP6: {len(drug_info_updated['ECFP6'].iloc[0])} (应为2048)")
print(f"MACCS: {len(drug_info_updated['MACCS'].iloc[0])} (应为168)")
print(f"Morgan: {len(drug_info_updated['Morgan'].iloc[0])} (应为1024)")


指纹长度验证:
ECFP2: 2048 (应为2048)
ECFP4: 2048 (应为2048)
ECFP6: 2048 (应为2048)
MACCS: 167 (应为168)
Morgan: 1024 (应为1024)


In [32]:
# ----- 5. check the Rdkit2D values 
def smiles_to_rdkit2d_cdf(smiles_list, n_quantiles=1000, return_df=True):
    """calculte 208-lenth RDKit 2D vector"""
    
    # 1. raw descriptor
    def _compute_descriptors(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return np.full(217, np.nan)
        return np.array(list(Descriptors.CalcMolDescriptors(mol).values()))
    
    raw_descriptors = np.array([_compute_descriptors(s) for s in smiles_list])
    
    # 2. missing values
    imputer = SimpleImputer(strategy='mean')
    imputed = imputer.fit_transform(raw_descriptors)
    
    # 3. CDF 
    cdf_transformer = QuantileTransformer(
        output_distribution='uniform',
        n_quantiles=min(len(smiles_list), n_quantiles)
    )
    cdf_normalized = cdf_transformer.fit_transform(imputed)
    
    # 4. results
    if return_df:
        return pd.DataFrame(cdf_normalized, 
                          columns=[f'RDKit2D_{i}' for i in range(217)],
                          index=range(len(smiles_list)))
    return cdf_normalized

# 返回DataFrame
# df_cdf = smiles_to_rdkit2d_cdf(drug_info_updated['Canonical SMILES'])
# print("DataFrame格式结果:")
# print(df_cdf.head())
drug_info_updated['Rdkit2D'] = drug_info_updated['Rdkit2D'].str.replace("，",",")
print(f"Rdkit2D: {len(drug_info_updated['Rdkit2D'].iloc[0].split(','))} (应为200)")

Rdkit2D: 200 (应为1024)


In [6]:
# ----- 6. remove missing data
drug_info_updated = drug_info_updated.dropna(subset=['Canonical SMILES', 'ECFP2'])
drug_info_updated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 918 entries, 0 to 1155
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Drugbank ID       918 non-null    object 
 1   PubChem CID       918 non-null    float64
 2   Canonical SMILES  918 non-null    object 
 3   InChIKey          917 non-null    object 
 4   ECFP2             918 non-null    object 
 5   ECFP4             918 non-null    object 
 6   ECFP6             918 non-null    object 
 7   MACCS             918 non-null    object 
 8   Morgan            918 non-null    object 
 9   Rdkit2D           918 non-null    object 
dtypes: float64(1), object(9)
memory usage: 78.9+ KB


In [7]:
drug_info_updated.to_csv('drugCidInfo_update.dat', sep='|', index=False, header=None)
drug_info_updated.iloc[:,[0,9]].to_csv('sider4_RDkit.dat', sep='|', index=False, header=None)
drug_info_updated.iloc[:,[0,9]].to_csv('sider4_Morgan.dat', sep='|', index=False, header=None)

In [142]:
# ----- 1.import ADR data 
adr_df = pd.read_csv("sider4_adr.csv")
adr_df.head()


,ID,C0002871,C0006142,C0006826,C0014175,C0025323,C0025874,C0027627,C0030193,C0042133,...,C1609538,C0038868,C0343386,C0026987,C0038002,C0004690,C0004691,C0282021,C0206062,C0241832
0,DB00014,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,DB00035,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,DB00050,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,DB00091,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,DB00104,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [143]:
# ----- 2.remove missing data
adr_df = adr_df[adr_df['ID'].isin(drug_info_updated['Drugbank ID'])]
adr_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 918 entries, 0 to 1192
Columns: 2083 entries, ID to C0241832
dtypes: int64(2082), object(1)
memory usage: 14.6+ MB


In [144]:
adr_df.iloc[:,1:].to_csv('sider4_adr.tsv', sep=',', index=False, header=None)

In [117]:
# ----- 1.import BIORDF data 
parent_path = os.path.dirname(os.getcwd())
Bio2RDF = pd.read_csv(os.path.join(parent_path,'Bio2RDF/Bio2RDFDrugFeature.txt'), sep='|',header=None)
Bio2RDF.columns = ["Drugbank ID","bio2rdf"]
Bio2RDF.head()


,Drugbank ID,bio2rdf
0,DB00808,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18..."
1,DB00809,"0,86,87,88,4,5,6,89,8,90,91,92,93,94,95,10,11,..."
2,DB00806,"0,133,134,135,136,137,138,139,4,5,89,8,140,9,1..."
3,DB00807,"0,196,4,6,8,95,11,197,14,17,18,99,24,29,198,31..."
4,DB06590,"0,133,235,236,4,5,6,89,8,140,237,238,92,93,95,..."


In [140]:
# ----- 2.remove missing data
bio2RDF_data = drug_info_updated[['Drugbank ID']].copy()
bio2RDF_data = bio2RDF_data.merge(Bio2RDF, on='Drugbank ID', how='left')
bio2RDF_data.info()
bio2RDF_data.to_csv("sider4_Bio2RDF.tsv", sep="|", index=False, header=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 918 entries, 0 to 917
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Drugbank ID  918 non-null    object
 1   bio2rdf      829 non-null    object
dtypes: object(2)
memory usage: 21.5+ KB


In [132]:
# bio2RDF_data["output"] = bio2RDF_data["Drugbank ID"]+"|"+bio2RDF_data["bio2rdf"].fillna('')
# joblib.dump(bio2RDF_data["output"].tolist(), "sider4_Bio2RDF.dat")

['sider4_Bio2RDF.dat']

In [133]:
# ----- 1.calculate ECFP descriptor
from openbabel import openbabel as ob
from openbabel import pybel
mol = ob.OBMol()
atom = mol.NewAtom()
print(dir(atom))

# (1) function test
import numpy as np

obConversion = ob.OBConversion()
obConversion.SetInAndOutFormats("smi", "mdl")

def simpleGenECFP(smile):
    mol = ob.OBMol()
    obConversion.ReadString(mol, smile)
    ecfp = []
    for atom in ob.OBMolAtomIter(mol):
        if atom.GetAtomicNum() == 1:
            continue
        atomECFP = np.ndarray(8, dtype=float)
        atomECFP[0] = atom.GetHvyDegree()
        atomECFP[1] = atom.GetExplicitValence()
        atomECFP[2] = atom.GetAtomicNum()
        atomECFP[3] = atom.GetIsotope()
        atomECFP[4] = atom.GetFormalCharge()
        atomECFP[5] = atom.ExplicitHydrogenCount() + atom.ImplicitHCount()
        atomECFP[6] = atom.IsInRing()
        atomECFP[7] = atom.IsAromatic()
        ecfp.append(atomECFP)
    ecfp = np.vstack(ecfp)
    return ecfp

def is_clockwise(atom):
    neighbors = list(atom.GetNeighbors())
    if len(neighbors) < 3:
        return False  # 无法判断立体化学方向

    atom_coord = np.array(atom.GetCoordinate())
    neighbor_coords = [np.array(neighbor.GetCoordinate()) for neighbor in neighbors]

    # 计算向量
    vectors = [coord - atom_coord for coord in neighbor_coords]

    # 取前三个向量计算叉积和点积
    v1, v2, v3 = vectors[0], vectors[1], vectors[2]
    cross_product = np.cross(v1, v2)
    dot_product = np.dot(cross_product, v3)

    # 如果点积为正，则方向为顺时针
    return dot_product > 0

def is_anti_clockwise(atom):
    return not is_clockwise(atom)

def is_clockwise(atom):
    neighbors = list(atom.GetNeighbors())
    if len(neighbors) < 3:
        return False  # 无法判断立体化学方向

    atom_coord = np.array(atom.GetCoordinate())
    neighbor_coords = [np.array(neighbor.GetCoordinate()) for neighbor in neighbors]

    # 计算向量
    vectors = [coord - atom_coord for coord in neighbor_coords]

    # 取前三个向量计算叉积和点积
    v1, v2, v3 = vectors[0], vectors[1], vectors[2]
    cross_product = np.cross(v1, v2)
    dot_product = np.dot(cross_product, v3)

    # 如果点积为正，则方向为顺时针
    return dot_product > 0

def is_anti_clockwise(atom):
    return not is_clockwise(atom)

def has_chiral_volume(atom):
    neighbors = list(atom.GetNeighbors())
    if len(neighbors) < 3:
        return False  # 至少需要三个邻接原子

    # 获取原子的坐标
    atom_coord = np.array(atom.GetCoordinate())
    neighbor_coords = [np.array(neighbor.GetCoordinate()) for neighbor in neighbors]

    # 计算所有邻接原子的向量
    vectors = [coord - atom_coord for coord in neighbor_coords]

    # 计算向量的标量三重积
    # 如果标量三重积为零，则说明原子共面，没有手性体积
    v1, v2, v3 = vectors[0], vectors[1], vectors[2]
    scalar_triple_product = np.dot(v1, np.cross(v2, v3))

    # 如果标量三重积不为零，则存在手性体积
    return scalar_triple_product != 0
    
def getAtomProperties(atom):
    atomProperties = [atom.GetFormalCharge(), 
                      atom.GetAtomicNum(), 
                      atom.GetIsotope(), 
                      atom.GetSpinMultiplicity(),
                      atom.GetAtomicMass(), 
                      atom.GetExactMass(), 
                      atom.GetExplicitDegree(), 
                      atom.GetHyb(),
                      atom.GetExplicitValence() + atom.GetImplicitHCount() - atom.GetFormalCharge(),
                      atom.GetHvyDegree(), 
                      atom.GetHeteroDegree(), 
                      atom.GetAtomicNum() == 1,
                      atom.GetAtomicNum() == 6,
                      atom.GetAtomicNum() == 7, 
                      atom.GetAtomicNum() == 8,
                      atom.GetAtomicNum() == 15, 
                      atom.GetAtomicNum() == 16,
                      atom.IsAromatic(), 
                      atom.IsInRing(), 
                      atom.IsInRingSize(5),
                      atom.IsInRingSize(6), 
                      atom.IsInRingSize(7), 
                      atom.IsInRingSize(8), 
                      atom.IsInRingSize(9),
                      atom.IsHeteroatom(), 
                      atom.GetAtomicNum() != 6 and atom.GetAtomicNum() != 1, 
                      atom.IsCarboxylOxygen(), 
                      atom.IsPhosphateOxygen(),
                      atom.IsSulfateOxygen(), 
                      atom.IsNitroOxygen(), 
                      atom.IsAmideNitrogen(), 
                      atom.IsPolarHydrogen(),
                      atom.IsNonPolarHydrogen(), 
                      atom.IsAromaticNOxide(), 
                      atom.IsChiral(), 
                      atom.IsAxial(),
                      #atom.IsClockwise(), 
                      #atom.IsAntiClockwise(), 
                      #atom.IsPositiveStereo(), 
                      #atom.IsNegativeStereo(),
                      #atom.HasChiralitySpecified(),
                      atom.IsChiral(),
                      #atom.HasChiralVolume(), 
                      atom.IsHbondAcceptor(), 
                      atom.IsHbondDonor(),
                      atom.IsHbondDonorH(), 
                      atom.IsMetal(), 
                      atom.HasNonSingleBond(), 
                      atom.HasSingleBond(),
                      atom.HasBondOfOrder(1), 
                      atom.HasBondOfOrder(2), 
                      atom.HasBondOfOrder(3), 
                      atom.HasAromaticBond()]
    try:
        atomProperties.extend([
            float(is_clockwise(atom)),  
            float(is_anti_clockwise(atom)),  
            float(is_positive_stereo(atom)),  
            float(is_negative_stereo(atom)),  
            float(has_chiral_volume(atom)),  
        ])
    except Exception as e:
        atomProperties.extend([0.0, 0.0, 0.0, 0.0, 0.0])

    size = len(atomProperties)
    ar = np.ndarray(size, dtype=float)
    for i in range(size):
        ar[i] = float(atomProperties[i])
    return ar


def fullPass1GenECFP(smile):
    mol = ob.OBMol()
    obConversion.ReadString(mol, smile)
    ecfp = []
    for atom in ob.OBMolAtomIter(mol):
        if atom.GetAtomicNum() == 1:
            continue
        atomECFP = getAtomProperties(atom)
        ecfp.append(atomECFP)
    ecfp = np.vstack(ecfp)
    return ecfp


if __name__ == "__main__":
    smile = "OCCN1CCN(CCCN2C3=CC=CC=C3SC3=C2C=C(Cl)C=C3)CC1"
    print(fullPass1GenECFP(smile))

['AddBond', 'AddResidue', 'AverageBondAngle', 'BeginBond', 'BeginBonds', 'BeginData', 'BeginNbrAtom', 'ClassDescription', 'Clear', 'ClearBond', 'ClearCoordPtr', 'CloneData', 'CountBondsOfOrder', 'CountFreeOxygens', 'CountFreeSulfurs', 'CountRingBonds', 'DataSize', 'DeleteBond', 'DeleteData', 'DeleteResidue', 'DoTransformations', 'Duplicate', 'EndBonds', 'EndData', 'ExplicitHydrogenCount', 'GetAllData', 'GetAngle', 'GetAtomicMass', 'GetAtomicNum', 'GetBond', 'GetCoordinate', 'GetCoordinateIdx', 'GetData', 'GetDistance', 'GetExactMass', 'GetExplicitDegree', 'GetExplicitValence', 'GetFormalCharge', 'GetHeteroDegree', 'GetHvyDegree', 'GetHyb', 'GetId', 'GetIdx', 'GetImplicitHCount', 'GetIndex', 'GetIsotope', 'GetNewBondVector', 'GetParent', 'GetPartialCharge', 'GetResidue', 'GetSpinMultiplicity', 'GetTitle', 'GetTotalDegree', 'GetTotalValence', 'GetType', 'GetVector', 'GetX', 'GetY', 'GetZ', 'HasAlphaBetaUnsat', 'HasAromaticBond', 'HasBondOfOrder', 'HasData', 'HasDoubleBond', 'HasNonSingle

In [136]:
def update_ecfp(df):
    df["ecfp"] = ""
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Updating ECFP"):
        smiles = row["Canonical SMILES"]
        if pd.notna(smiles):  
            df.at[index,"ecfp"] = fullPass1GenECFP(smiles)
        else:
            df.at[index,"ecfp"] = ""
    return df

ecfp_data = update_ecfp(drug_info_updated.loc[:,["Drugbank ID","Canonical SMILES"]])
ecfp_data.head()

Updating ECFP: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 918/918 [00:00<00:00, 1068.15it/s]


,Drugbank ID,Canonical SMILES,ecfp
0,DB00014,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,"[[0.0, 6.0, 0.0, 0.0, 12.0107, 12.0, 1.0, 3.0,..."
3,DB00091,C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC...,"[[0.0, 6.0, 0.0, 0.0, 12.0107, 12.0, 1.0, 3.0,..."
5,DB00125,N=C(N)NCCC[C@H](N)C(=O)O,"[[0.0, 7.0, 0.0, 0.0, 14.0067, 14.003074005, 1..."
6,DB00132,CC/C=C\C/C=C\C/C=C\CCCCCCCC(=O)O,"[[0.0, 6.0, 0.0, 0.0, 12.0107, 12.0, 1.0, 3.0,..."
7,DB00136,C=C1/C(=C\C=C2/CCC[C@@]3(C)[C@H]2CC[C@@H]3[C@H...,"[[0.0, 6.0, 0.0, 0.0, 12.0107, 12.0, 1.0, 2.0,..."


In [138]:
def check_data_structure(data, depth=0):

    if isinstance(data, list):
        print(f"{'  ' * depth}List with {len(data)} elements:")

        for i, element in enumerate(data[:3]): 
            print(f"{'  ' * (depth + 1)}Element {i}:")
            check_data_structure(element, depth + 2)

    elif isinstance(data, np.ndarray):
        print(f"{'  ' * depth}ndarray with shape {data.shape}:")

        if data.size > 10: 
            print(f"{'  ' * (depth + 1)}{data[:5, :5]}...") 
        else:
            print(f"{'  ' * (depth + 1)}{data}")

    elif isinstance(data, dict):
        print(f"{'  ' * depth}Dict with {len(data)} keys: {list(data.keys())[:3]}")
        for key, value in list(data.items())[:3]: 
            print(f"{'  ' * (depth + 1)}Key={key}")
            check_data_structure(value, depth + 2)

    else:
        print(f"{'  ' * depth}{type(data).__name__}: {data}")
        
check_data_structure(ecfp_data["ecfp"].tolist(),3)

N_DRUGS = len(ecfp_data["ecfp"].tolist())
N_FEATURE = len(ecfp_data["ecfp"].tolist()[0][0])
MAX_ATOMS = max(len(sublist) for sublist in ecfp_data["ecfp"].tolist())
print("N_DRUGS = {}, N_FEATURES = {}, MAX_ATOMS = {}".format(N_DRUGS, N_FEATURE, MAX_ATOMS))

      List with 918 elements:
        Element 0:
          ndarray with shape (91, 52):
            [[ 0.      6.      0.      0.     12.0107]
 [ 0.      6.      0.      0.     12.0107]
 [ 0.      6.      0.      0.     12.0107]
 [ 0.      6.      0.      0.     12.0107]
 [ 0.      6.      0.      0.     12.0107]]...
        Element 1:
          ndarray with shape (85, 52):
            [[ 0.      6.      0.      0.     12.0107]
 [ 0.      6.      0.      0.     12.0107]
 [ 0.      6.      0.      0.     12.0107]
 [ 0.      6.      0.      0.     12.0107]
 [ 0.      6.      0.      0.     12.0107]]...
        Element 2:
          ndarray with shape (12, 52):
            [[ 0.      7.      0.      0.     14.0067]
 [ 0.      6.      0.      0.     12.0107]
 [ 0.      7.      0.      0.     14.0067]
 [ 0.      7.      0.      0.     14.0067]
 [ 0.      6.      0.      0.     12.0107]]...
N_DRUGS = 918, N_FEATURES = 52, MAX_ATOMS = 101


In [139]:
joblib.dump(ecfp_data["ecfp"].tolist(), "sider4_ecfp.dat")

['sider4_ecfp.dat']